In [2]:
import pandas as pd
import numpy as np
import matplotlib
from tensorflow import keras
import os
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt 
from sklearn.preprocessing import MinMaxScaler 

# read data set
base_src = './drive/MyDrive/machine_learning/datasets/codebasics'
friend_src = base_src + '/insurance_data.csv'


FileNotFoundError: ignored

In [6]:
df = pd.read_csv(friend_src)
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2, random_state=25)

In [8]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [9]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), 
                       activation='sigmoid', 
                       # 2개의 뉴런을 두고 
                       # 두 weight 모두 1
                      # bias 는 0 으로 세팅해둔다
                       kernel_initializer='ones', 
                       bias_initializer='zeros')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1/1 [==============================] - 0s 8ms/step - loss: 0.5321 - accuracy: 0.7273
Epoch 2502/5000
1/1 [==============================] - 0s 5ms/step - loss: 0.5321 - accuracy: 0.7273
Epoch 2503/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.5321 - accuracy: 0.7273
Epoch 2504/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.5320 - accuracy: 0.7273
Epoch 2505/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.5320 - accuracy: 0.7273
Epoch 2506/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.5320 - accuracy: 0.7273
Epoch 2507/5000
1/1 [==============================] - 0s 24ms/step - loss: 0.5319 - accuracy: 0.7273
Epoch 2508/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.5319 - accuracy: 0.7273
Epoch 2509/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.5318 - accuracy: 0.7273
Epoch 2510/5000
1/1 [==============================] - 0s 7ms/step - l

In [ ]:
# 시간 매우 오래 걸릴 것
model.fit(X_train_scaled, y_train, epochs=5000)

In [10]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 195ms/step - loss: 0.3550 - accuracy: 1.0000


[0.35497748851776123, 1.0]

In [11]:
# final weight, bias 를 알아내고 싶다
coef, intercept = model.get_weights()
coef, intercept

(array([[5.060867 ],
        [1.4086502]], dtype=float32), array([-2.9137027], dtype=float32))

In [13]:
def sigmoid(x) :
  import math
  return 1 / (1 + math.exp(x))
sigmoid(10)

4.5397868702434395e-05

In [17]:
def prediction_function(age,affordibility) :
  weight_sum = coef[0]* age + coef[1] * affordibility + intercept
  return sigmoid(weight_sum)

In [18]:
prediction_function(0.47,1)

0.29451513068638835

In [26]:
# apply gradient descent
# Log loss
epsilon = 1e-15
def log_loss(y_true, y_predicted):
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [24]:
def sigmoid_numpy(X) :
  return 1 / (1+np.exp(-X))

In [28]:
def gradient_descent(age,affordibility,y_true,epoche,loss_threshold) :
  # w1,w2,bias
  w1 = w2 = 1
  bias    = 0
  rate    = 0.5
  n       = len(age)
  for i in range(epoche):
    weighted_sum = w1 * age + w2 * affordibility + bias  # find weighted sum
    y_predicted = sigmoid_numpy(weighted_sum)
    loss = log_loss(y_true,y_predicted)

    # update weight based on derivative function we have
    w1d    = (1/n) * np.dot((y_predicted - y_true) ,np.transpose(age))
    w2d    = (1/n) * np.dot((y_predicted - y_true) ,np.transpose(affordibility))
    
    bias_d = (1/n) * np.dot((y_predicted - y_true) ,np.transpose(bias))

    w1 = w1 - rate * w1d
    w2 = w2 - rate * w2d
    bias = bias - rate * bias_d

    print(f'Epoch:{i}, w1 : {w1} , w2 : {w2}, bias : {bias} , loss : {loss} ')

    if loss <= loss_threshold : break
  
  return w1,w2,bias

In [29]:
gradient_descent(X_train_scaled['age'],X_train_scaled['affordibility'],y_train, 1000, 0.4631) 
# 0.4631이 최종값 --> loss 값 변화가 크게 없다면 더이상 출력 x 위해서 

Epoch:0, w1 : 0.974907633470177 , w2 : 0.948348125394529, bias : [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] , loss : 0.7113403233723417 
Epoch:1, w1 : 0.951309050414721 , w2 : 0.8998384766138976, bias : [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] , loss : 0.7049437880152859 
Epoch:2, w1 : 0.9291619684235712 , w2 : 0.8543880770572044, bias : [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] , loss : 0.6993052593238981 
Epoch:3, w1 : 0.9084179079556838 , w2 : 0.8118995045564921, bias : [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] , loss : 0.6943576384281476 
Epoch:4, w1 : 0.8890233255496454 , w2 : 0.7722634615046652, bias : [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] , loss : 0.6900349522584058 
Epoch:5, w1 : 0.870920747167955 , w2 : 0.7353613402471417, bias : [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] , loss : 0.6862734824669942 
Epoch:6, w1 : 0.8540

(0.5738399872440308,
 0.34449479487147006,
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]))